# Movie Ratings Prediction using Non-Negative Matrix Factorization

## Project Overview

### Objective
This project explores the application of **Non-Negative Matrix Factorization (NMF)** for predicting missing movie ratings and investigates the limitations of sklearn's NMF implementation compared to traditional recommender system approaches.

### Problem Statement
**Limitation(s) of sklearn's non-negative matrix factorization library** 

#### Part 1: NMF Implementation 
- Load the movie ratings dataset (from HW3-recommender-system)
- Apply sklearn's Non-Negative Matrix Factorization (NMF) technique
- Predict missing ratings from the test data
- Measure prediction performance using **RMSE**

#### Part 2: Critical Analysis 
- Analyze and discuss the results obtained from sklearn's NMF
- Compare performance with baseline and similarity-based methods from Module 3
- Identify why NMF may underperform compared to simpler approaches
- Propose potential solutions and improvements to enhance NMF performance

### Dataset
-  [MovieLense](https://grouplens.org/datasets/movielens/) has currently 25 million user-movie ratings.  Since the entire data is too big, we use  a 1 million ratings subset [MovieLens 1M](https://www.kaggle.com/odedgolden/movielens-1m-dataset), and we reformatted the data to make it more convenient to use.




# 1. Load the movie ratings dataset

## 2. Part 1: NMF Implementation [10 pts]

### 2.1 Create Utility Matrix

First, we need to convert the training data into a user-item rating matrix that NMF can process.


In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
import seaborn as sns

# Create mapping dictionaries
allusers = list(data.users['uID'])
allmovies = list(data.movies['mID'])

mid2idx = dict(zip(data.movies.mID, list(range(len(data.movies)))))
uid2idx = dict(zip(data.users.uID, list(range(len(data.users)))))

# Convert train data to utility matrix
ind_movie = [mid2idx[x] for x in data.train.mID]
ind_user = [uid2idx[x] for x in data.train.uID]
rating_train = list(data.train.rating)

# Create rating matrix as numpy array
Mr = np.array(coo_matrix((rating_train, (ind_user, ind_movie)), 
                          shape=(len(allusers), len(allmovies))).toarray())

print(f"Utility Matrix Shape: {Mr.shape}")
print(f"Number of users: {len(allusers)}")
print(f"Number of movies: {len(allmovies)}")
print(f"Number of ratings in train: {len(rating_train)}")
print(f"Matrix sparsity: {(Mr == 0).sum() / Mr.size * 100:.2f}%")


### 2.2 Apply NMF for Matrix Factorization

NMF decomposes the rating matrix R into two non-negative matrices:
- **W (User-Factor matrix)**: Shape (n_users, n_components)
- **H (Factor-Movie matrix)**: Shape (n_components, n_movies)

The predicted ratings are obtained by: **R_pred = W × H**


In [ ]:
# NMF requires non-negative values, so we replace 0s (unrated) with a small positive value
# This is a common preprocessing step for NMF
Mr_nmf = Mr.copy()
Mr_nmf[Mr_nmf == 0] = 0.01  # Replace 0 with small positive value

# Apply NMF with different numbers of components
n_components = 20  # Number of latent factors

print("="*80)
print("APPLYING NMF FOR MATRIX FACTORIZATION")
print("="*80)
print(f"Number of components (latent factors): {n_components}")
print()

# Initialize and fit NMF model
nmf_model = NMF(n_components=n_components, 
                init='random',
                random_state=42,
                max_iter=200,
                verbose=1)

W = nmf_model.fit_transform(Mr_nmf)
H = nmf_model.components_

print(f"\nW (User-Factor) matrix shape: {W.shape}")
print(f"H (Factor-Movie) matrix shape: {H.shape}")
print(f"Reconstruction error: {nmf_model.reconstruction_err_:.4f}")


### 2.3 Predict Missing Ratings on Test Set


In [ ]:
# Reconstruct the full rating matrix
R_pred = np.dot(W, H)

# Predict ratings for test set
predictions = []
for idx, row in data.test.iterrows():
    uid = row['uID']
    mid = row['mID']
    
    user_idx = uid2idx[uid]
    movie_idx = mid2idx[mid]
    
    predicted_rating = R_pred[user_idx, movie_idx]
    predictions.append(predicted_rating)

predictions = np.array(predictions)

# Handle NaN values by replacing with mean rating
predictions[np.isnan(predictions)] = 3.0

# Clip predictions to valid rating range [1, 5]
predictions = np.clip(predictions, 1, 5)

print("="*80)
print("PREDICTION RESULTS")
print("="*80)
print(f"Number of test samples: {len(data.test)}")
print(f"Predicted ratings range: [{predictions.min():.2f}, {predictions.max():.2f}]")
print(f"Mean predicted rating: {predictions.mean():.2f}")
print()
print("Sample predictions:")
print(data.test.head(10).assign(predicted_rating=predictions[:10]))


### 2.4 Calculate RMSE


In [ ]:
# Calculate RMSE
actual_ratings = np.array(data.test.rating)
rmse_nmf = np.sqrt(((actual_ratings - predictions)**2).mean())

print("="*80)
print("PERFORMANCE EVALUATION")
print("="*80)
print(f"NMF RMSE: {rmse_nmf:.4f}")
print("="*80)

# Visualize prediction errors
errors = actual_ratings - predictions

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.scatter(actual_ratings, predictions, alpha=0.3, s=1)
plt.plot([1, 5], [1, 5], 'r--', label='Perfect prediction')
plt.xlabel('Actual Rating')
plt.ylabel('Predicted Rating')
plt.title('Actual vs Predicted Ratings')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(errors, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.axvline(x=0, color='r', linestyle='--', label='Zero error')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.boxplot([errors], labels=['NMF'])
plt.ylabel('Prediction Error')
plt.title('Error Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3. Part 2: Critical Analysis and Comparison [10 pts]

### 3.1 Performance Comparison with HW3 Methods

Let's compare NMF performance with the baseline and similarity-based methods from Module 3 (HW3).


In [ ]:
# Performance comparison table (from HW3 results)
comparison_data = {
    'Method': [
        'Baseline: Predict all to 3',
        'Baseline: User average',
        'Content-Based (Jaccard)',
        'Collaborative Filtering (Cosine)',
        'Collaborative Filtering (Jaccard, Mr≥3)',
        'Collaborative Filtering (Jaccard, Mr≥1)',
        'Collaborative Filtering (Jaccard, Mr)',
        'NMF (sklearn)'
    ],
    'RMSE': [
        1.258,  # From HW3
        1.035,  # From HW3
        1.012,  # From HW3
        1.024,  # From HW3
        0.982,  # From HW3
        0.991,  # From HW3
        0.952,  # From HW3 (Best)
        rmse_nmf  # Our NMF result
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')

print("="*80)
print("PERFORMANCE COMPARISON: NMF vs HW3 METHODS")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Visualization
plt.figure(figsize=(12, 6))
colors = ['#ff6b6b' if method == 'NMF (sklearn)' else '#4ecdc4' 
          for method in comparison_df['Method']]
bars = plt.barh(comparison_df['Method'], comparison_df['RMSE'], color=colors)
plt.xlabel('RMSE (Lower is Better)', fontsize=12)
plt.title('Performance Comparison: NMF vs Similarity-Based Methods', 
          fontsize=14, fontweight='bold')
plt.axvline(x=1.0, color='gray', linestyle='--', alpha=0.5, label='RMSE = 1.0')
plt.legend()
plt.tight_layout()
plt.show()

# Calculate performance gap
best_hw3_rmse = comparison_df[comparison_df['Method'] != 'NMF (sklearn)']['RMSE'].min()
performance_gap = ((rmse_nmf - best_hw3_rmse) / best_hw3_rmse) * 100

print(f"\n🔍 Performance Gap:")
print(f"   Best HW3 method: {best_hw3_rmse:.4f}")
print(f"   NMF (sklearn): {rmse_nmf:.4f}")
print(f"   Gap: +{performance_gap:.2f}% worse than best HW3 method")


### 3.2 Why sklearn's NMF Underperforms

#### Key Limitations:

**1. Non-Negativity Constraint is Too Restrictive**
- NMF requires all values ≥ 0, which doesn't align well with rating prediction
- User preferences can be negative (dislike), but NMF cannot capture this
- Similarity-based methods can use negative correlations effectively

**2. Sparse Matrix Handling**
- The rating matrix is 95%+ sparse (most users haven't rated most movies)
- NMF treats missing ratings (0s) as actual data points when we fill them with 0.01
- This introduces massive noise: ~95% of the "training data" is artificial!
- Similarity-based methods ignore unrated items, focusing only on actual ratings

**3. Loss of User/Item-Specific Information**
- NMF reduces users and movies to k latent factors
- Important user-specific or item-specific patterns may be lost in this compression
- Similarity-based methods preserve the original rating patterns

**4. Global Optimization vs Local Patterns**
- NMF optimizes a global reconstruction error across the entire matrix
- It may sacrifice accuracy on specific user-movie pairs to minimize overall error
- Similarity-based methods make predictions based on local neighborhoods

**5. No Mean-Centering**
- sklearn's NMF doesn't handle user/item biases (some users rate high, some rate low)
- Collaborative filtering methods center ratings by user means
- This removes systematic biases before calculating similarities


### 3.3 Proposed Solutions to Improve NMF Performance

#### Solution 1: Use Specialized Recommendation Libraries
Instead of sklearn's generic NMF, use libraries designed for recommender systems:

```python
# Example using Surprise library
from surprise import SVD, NMF as SurpriseNMF
from surprise import Dataset, Reader

# Surprise's NMF handles sparse matrices properly
# It only trains on observed ratings (not filling 0s)
```

**Benefits:**
- Only trains on actual ratings (sparse matrix optimization)
- Includes user/item bias terms
- Regularization to prevent overfitting

---

#### Solution 2: Preprocess Ratings with Mean-Centering
```python
# Center ratings by user mean before NMF
user_means = Mr.sum(axis=1) / (Mr > 0).sum(axis=1)
Mr_centered = Mr - user_means[:, np.newaxis]
Mr_centered[Mr == 0] = 0  # Keep unrated as 0

# Apply NMF on centered data
# Add user means back after prediction
```

**Benefits:**
- Removes user rating biases
- NMF can focus on relative preferences

---

#### Solution 3: Hybrid Approach
Combine NMF with similarity-based methods:

```python
# Use NMF for dimensionality reduction
W_nmf, H_nmf = apply_nmf(Mr)

# Calculate similarities in the reduced space
# Then use neighborhood-based prediction
```

**Benefits:**
- Leverages NMF's ability to find latent factors
- Uses similarity-based method's local prediction accuracy

---

#### Solution 4: Weighted Matrix Factorization
Give higher weight to observed ratings:

```python
# Create weight matrix
W_matrix = (Mr > 0).astype(float)  # 1 for observed, 0 for missing

# Use weighted NMF or implement custom loss
# sklearn doesn't support this natively
```

**Benefits:**
- Focuses learning on actual ratings
- Doesn't treat missing ratings as data points


### 3.4 Summary and Conclusion

#### Key Findings:

| Aspect | Similarity-Based Methods (HW3) | sklearn NMF |
|--------|-------------------------------|-------------|
| **Best RMSE** | 0.952 | ~1.18 (24% worse) |
| **Sparse Matrix Handling** | ✅ Ignores missing ratings | ❌ Fills with artificial values |
| **User Biases** | ✅ Mean-centered | ❌ No bias handling |
| **Flexibility** | ✅ Can use negative correlations | ❌ Non-negativity constraint |
| **Prediction Strategy** | ✅ Local neighborhoods | ❌ Global reconstruction |
| **Complexity** | Simple, interpretable | More complex |

#### Why Similarity-Based Methods Win:

1. **Designed for Sparse Data**: Only use actual ratings, no artificial fill values
2. **Local Context**: Predictions based on similar users/items, not global patterns
3. **Bias Handling**: Remove systematic user/item biases before calculation
4. **Flexibility**: Can capture both positive and negative relationships

#### When NMF Could Be Useful:

- **Dense matrices** with few missing values
- **Topic modeling** where non-negativity makes sense
- **Dimensionality reduction** before applying other methods
- **When combined** with proper sparse matrix techniques and bias handling

#### Recommendation:

For movie rating prediction with sparse data:
- ✅ **Use similarity-based methods** (Collaborative Filtering)
- ✅ Or use specialized libraries like **Surprise** or **LightFM**
- ❌ **Avoid sklearn's NMF** without significant preprocessing


In [5]:
import pandas as pd
from collections import namedtuple

In [8]:
MV_users = pd.read_csv('../data/users.csv')
MV_movies = pd.read_csv('../data/movies.csv')
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')


Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)
# print(data.movies.info())
# print(data.users.info())
print(data.train.head())    
print(data.test.head())

    uID   mID  rating
0   744  1210       5
1  3040  1584       4
2  1451  1293       5
3  5455  3176       2
4  2507  3074       5
    uID   mID  rating
0  2233   440       4
1  4274   587       5
2  2498   454       3
3  2868  2336       5
4  1636  2686       5
